##### Import

In [45]:
import pandas as pd 
import os
from pathlib import Path
import numpy as np

##### Crée un dictionnaire avec tous les datasets 

In [2]:
csvDict = {}

for subdir, dirs, files in os.walk(r'Datasets_Kaggle/'):
    for filename in files:
        filepath = subdir + os.sep + filename
        if filepath.endswith(".csv"):
            #if "MN5_AROME" in filename:
            csvDict[filename[:-4]] = pd.read_csv(Path(filepath),sep=",")

for i in csvDict.keys():
    print (i)

circuits
constructors
constructor_results
constructor_standings
drivers
driver_standings
lap_times
pit_stops
qualifying
races
results
status


##### Affiche le nom de toutes les colonnes de tous les datasets

In [3]:
for dataset in csvDict:
    print("---", dataset)
    for col in csvDict[dataset]:
        print(col)

--- circuits
circuitId
circuitRef
name
location
country
lat
lng
alt
url
--- constructors
constructorId
constructorRef
name
nationality
url
--- constructor_results
constructorResultsId
raceId
constructorId
points
status
--- drivers
driverId
driverRef
number
code
forename
surname
dob
nationality
url
--- lap_times
raceId
driverId
lap
position
time
milliseconds
--- pit_stops
raceId
driverId
stop
lap
time
duration
milliseconds
--- qualifying
qualifyId
raceId
driverId
constructorId
number
position
q1
q2
q3
--- races
raceId
year
round
circuitId
name
date
time
url
--- results
resultId
raceId
driverId
constructorId
number
grid
position
positionText
positionOrder
points
laps
time
milliseconds
fastestLap
rank
fastestLapTime
fastestLapSpeed
statusId
--- status
statusId
status


##### Pour visualiser un dataset parmi le dictionnaire de datasets

In [4]:
csvDict["races"].head()

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


##### Vérifie que les dates soient bien au format date compris par la librairie pandas

In [5]:
csvDict["races"]["date"] = pd.to_datetime(csvDict["races"]["date"])

##### Choses à faire dans le nettoyage (à première vue)

- Se poser la question d'utiliser la totalité des données historique que l'on a à disposition
- Supprimer les URL si nécessaire 
---
- Voir au niveau des écuries constructeurs si ce ne sont pas les mêmes car parfois il y a juste un changement de nom d'écurie mais c'est toujours la même (dataset constructors)
- Lier les résultats des constructeurs au dataset constructors (voir comment faire, éventuellement un dataset pour chaque écurie avec ses résultats par courses)
- Voir si il y a une différence entre position et positionText (dataset constructor_standings)
- Voir la différence de lignes à quoi elle correspond entre le dataset constructor_results et constructor_standings
- Lier les constructeurs aux résultats obtenus par leur pilote avec leur statusID (présent dans le dataset status)
---
- Faire le lien entre drivers et driver_standings
- Lier les pilotes à leur résultat en qualification
- Lier les pilotes à leur résultat en course avec leur statusID (présent dans le dataset status)
---
- Voir quel intérêt dans un modèle de Machine Learning il peut y avoir à étudier le temps de chaque pilote à chaque tour par course (sachant que dans le dataset results, on a déjà les meilleurs temps de chaque pilote de la course)
- Voir quel intérêt en ML de conserver les pit_stops
---
- Lier les datasets circuits et races
---
- seasons ? utilité de conserver ce dataset ? 

In [51]:
csvDict["results"].head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [74]:
 csvDict["status"].status.unique()

array(['Finished', 'Disqualified', 'Accident', 'Collision', 'Engine',
       'Gearbox', 'Transmission', 'Clutch', 'Hydraulics', 'Electrical',
       '+1 Lap', '+2 Laps', '+3 Laps', '+4 Laps', '+5 Laps', '+6 Laps',
       '+7 Laps', '+8 Laps', '+9 Laps', 'Spun off', 'Radiator',
       'Suspension', 'Brakes', 'Differential', 'Overheating',
       'Mechanical', 'Tyre', 'Driver Seat', 'Puncture', 'Driveshaft',
       'Retired', 'Fuel pressure', 'Front wing', 'Water pressure',
       'Refuelling', 'Wheel', 'Throttle', 'Steering', 'Technical',
       'Electronics', 'Broken wing', 'Heat shield fire', 'Exhaust',
       'Oil leak', '+11 Laps', 'Wheel rim', 'Water leak', 'Fuel pump',
       'Track rod', '+17 Laps', 'Oil pressure', '+42 Laps', '+13 Laps',
       'Withdrew', '+12 Laps', 'Engine fire', 'Engine misfire',
       '+26 Laps', 'Tyre puncture', 'Out of fuel', 'Wheel nut',
       'Not classified', 'Pneumatics', 'Handling', 'Rear wing', 'Fire',
       'Wheel bearing', 'Physical', 'Fuel sys

In [61]:
a = csvDict["results"].loc[csvDict["results"]["fastestLap"] == '\\N']
a

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
16,17,18,17,9,10,14,\N,R,17,0.0,0,\N,\N,\N,\N,\N,\N,4
17,18,18,18,11,16,12,\N,R,18,0.0,0,\N,\N,\N,\N,\N,\N,4
18,19,18,19,8,19,21,\N,R,19,0.0,0,\N,\N,\N,\N,\N,\N,4
19,20,18,20,5,15,9,\N,R,20,0.0,0,\N,\N,\N,\N,\N,\N,4
20,21,18,21,10,21,16,\N,R,21,0.0,0,\N,\N,\N,\N,\N,\N,4
42,43,19,10,7,12,10,\N,R,21,0.0,1,\N,\N,\N,\N,\N,\N,4
43,44,19,7,5,14,18,\N,R,22,0.0,0,\N,\N,\N,\N,\N,\N,20
65,66,20,20,5,15,19,\N,R,22,0.0,0,\N,\N,\N,\N,\N,\N,5
86,87,21,16,10,20,20,\N,R,21,0.0,0,\N,\N,\N,\N,\N,\N,4
87,88,21,20,5,15,18,\N,R,22,0.0,0,\N,\N,\N,\N,\N,\N,4


In [62]:
csvDict["results"] = csvDict["results"].drop(columns = ["position","positionText","milliseconds"], axis = 1)

In [63]:
csvDict["results"].head()

,resultId,raceId,driverId,constructorId,number,grid,positionOrder,points,laps,time,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,10.0,58,1:34:50.616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,8.0,58,+5.478,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,6.0,58,+8.163,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,5.0,58,+17.181,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,4.0,58,+18.014,43,1,1:27.418,218.385,1


In [68]:
csvDict["results"]["rank"] = csvDict["results"]["rank"].replace('\\N', np.nan )
csvDict["results"]

,resultId,raceId,driverId,constructorId,number,grid,positionOrder,points,laps,time,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,10.0,58,1:34:50.616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,8.0,58,+5.478,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,6.0,58,+8.163,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,5.0,58,+17.181,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,4.0,58,+18.014,43,1,1:27.418,218.385,1
5,6,18,6,3,8,13,6,3.0,57,NaN,50,14,1:29.639,212.974,11
6,7,18,7,5,14,17,7,2.0,55,NaN,22,12,1:29.534,213.224,5
7,8,18,8,6,1,15,8,1.0,53,NaN,20,4,1:27.903,217.180,5
8,9,18,9,2,4,2,9,0.0,47,NaN,15,9,1:28.753,215.100,4
9,10,18,10,7,12,18,10,0.0,43,NaN,23,13,1:29.558,213.166,3


In [71]:
csvDict["results"] = csvDict["results"].rename(columns = {"rank" : "rankFastestLap"})
csvDict["results"]

,resultId,raceId,driverId,constructorId,number,grid,positionOrder,points,laps,time,fastestLap,rankFastestLap,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,10.0,58,1:34:50.616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,8.0,58,+5.478,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,6.0,58,+8.163,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,5.0,58,+17.181,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,4.0,58,+18.014,43,1,1:27.418,218.385,1
5,6,18,6,3,8,13,6,3.0,57,NaN,50,14,1:29.639,212.974,11
6,7,18,7,5,14,17,7,2.0,55,NaN,22,12,1:29.534,213.224,5
7,8,18,8,6,1,15,8,1.0,53,NaN,20,4,1:27.903,217.180,5
8,9,18,9,2,4,2,9,0.0,47,NaN,15,9,1:28.753,215.100,4
9,10,18,10,7,12,18,10,0.0,43,NaN,23,13,1:29.558,213.166,3


In [72]:
csvDict["results"].to_csv("results_Clean.csv",sep = ";")

In [73]:
csvDict["status"].to_csv("Data_Clean/status_Clean.csv",sep = ";")